In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
n_element = 10

element_score = pd.Series(
    np.arange(n_element),
    name="Score",
    index=pd.Index(("Element{}".format(i) for i in range(n_element)), name="Element"),
)

set_elements = element_score.index[-2:]

for input_ in (
    "rank",
    #     "score",
):

    for distribution in (
        #         "pdf",
        "cdf",
    ):

        for signal in (
            "s0",
            #             "s1",
            #             "s2",
        ):

            for score in (
                "supreme",
                #                 "area",
            ):

                method = (input_, distribution, signal, score)

                kraft.score_set(element_score, set_elements, method=method, plot_=True)

In [ ]:
for n_element in (100,):

    element_score = np.random.normal(size=n_element // 2)

    element_score = pd.Series(
        np.sort(np.concatenate((-element_score, element_score))),
        name="Score",
        index=pd.Index(
            ("Element{}".format(i) for i in range(n_element)), name="Element"
        ),
    )

    for n_skip in (0.1,):

        if n_skip < 1:

            n_skip = int(n_skip * n_element)

        for n_top in (1,):

            if n_top < 1:

                n_top = int(n_top * n_element)

            for rate_top_hit in (1,):

                if rate_top_hit < 1:

                    rate_top_hit = int(rate_top_hit * n_top)

                for side in ("+",):

                    set_elements = []

                    if "-" in side:

                        set_elements += element_score.index[
                            n_skip : n_skip + n_top : rate_top_hit
                        ].tolist()

                    if "+" in side:

                        set_elements += element_score.index[
                            -(n_skip + 1) : -(n_skip + 1 + n_top) : -rate_top_hit
                        ].tolist()

                    for method in ("rank cdf s0 supreme",):

                        kraft.score_set(
                            element_score, set_elements, method=method, plot_=True,
                        )

In [ ]:
element_score = pd.read_csv("gene_score.tsv", sep="\t", index_col=0, squeeze=True)

set_elements = kraft.read_gmt("c2.gmt")["COLLER_MYC_TARGETS_UP"]

In [ ]:
kraft.score_set(element_score, set_elements, method="rank cdf s0 supreme", plot_=True)